In [3]:
import os
import anndata as ad
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
correspondence = {}
correspondence['149Sm_CREB'] ='149Sm_pCREB'
correspondence['167Er_ERK'] ='167Er_pERK12'
correspondence['164Dy_IkB'] ='164Dy_IkB'
correspondence['159Tb_MAPKAPK2'] ='159Tb_pMAPKAPK2'
correspondence['166Er_NFkB'] ='166Er_pNFkB'
correspondence['151Eu_p38'] ='151Eu_pp38'
correspondence['155Gd_S6'] ='155Gd_pS6'
correspondence['153Eu_STAT1'] ='153Eu_pSTAT1'
correspondence['154Sm_STAT3'] ='154Sm_pSTAT3'
correspondence['150Nd_STAT5'] ='150Nd_pSTAT5'
correspondence['168Er_pSTAT6'] ='168Yb_pSTAT6'
correspondence['174Yb_HLADR'] ='174Yb_HLADR'
correspondence['169Tm_CD25'] ='169Tm_CD25'

In [16]:
def create_density_plots(dist_data, out_file, title_suffix=""):
    sns.set_theme(style="whitegrid")
    pts_sorted = sorted(dist_data.keys())

    num_plots = len(pts_sorted)
    cols = min(3, num_plots)
    rows = int(np.ceil(num_plots / cols))

    fig_width = max(5 * cols, 8)
    fig_height = 5 * rows
    fig, axes = plt.subplots(
        rows, cols, figsize=(fig_width, fig_height), constrained_layout=True
    )


    if num_plots == 1:
        axes = np.array([axes])

    cat_labels = ["Unstim surge", 'Stim True surge',"Stim pred surge"]
    cat_colors = ["blue", "red", "green"]

    for i, (pt, ax) in enumerate(zip(pts_sorted, axes.flatten())):
        for label, color in zip(cat_labels, cat_colors):
            arr = dist_data[pt][label]
            if arr.size > 0:
                sns.kdeplot(
                    arr,
                    ax=ax,
                    label=f"{label} (n={arr.size})",
                    color=color,
                    fill=False,  # set tot True to fill the area under the curve
                    alpha=0.3,
                )

        ax.set_title(f"Surge corrected prediction:", fontsize=14)
        ax.set_xlabel("Value", fontsize=12)
        ax.set_ylabel("Density", fontsize=12)
        ax.legend(fontsize=10)
        ax.grid(True)
    for j in range(i + 1, len(axes.flatten())):
        fig.delaxes(axes.flatten()[j])

    plt.savefig(out_file, dpi=200, bbox_inches="tight")
    plt.close()


def plot_result(path_cohort_1, marker, outdir_path,doms_stim):
    cohort1 = ad.read(path_cohort_1)
    cohort1 = cohort1[:, marker].copy()
    #doms_stim='P. gingivalis'
    unstim = pd.Series(
        cohort1[cohort1.obs["drug"] == "Unstim"].X.flatten(), name="Unstim surge"
    )
    stim_true = pd.Series(
        cohort1[(cohort1.obs["drug"] ==doms_stim) & (cohort1.obs["state"] =='true_corrected')].X.flatten(), name="Stim True surge"
    )
    stim_pred=pd.Series(
        cohort1[(cohort1.obs["drug"] ==doms_stim) & (cohort1.obs["state"] =='predicted')].X.flatten(), name="Stim pred surge"
    )
    
    dist_data = {
        "Patient_1": {
            "Unstim surge": unstim.values,
            "Stim True surge": stim_true.values,
            "Stim pred surge": stim_pred.values,
        }
    }

    create_density_plots(dist_data, outdir_path, title_suffix="")
    return


In [17]:
marker_list=['149Sm_pCREB', '167Er_pERK12', '164Dy_IkB', '159Tb_pMAPKAPK2',
       '166Er_pNFkB', '151Eu_pp38', '155Gd_pS6', '153Eu_pSTAT1',
       '154Sm_pSTAT3', '150Nd_pSTAT5', '168Yb_pSTAT6', '174Yb_HLADR',
       '169Tm_CD25']
features2 = ['149Sm_pCREB','159Tb_pMAPKAPK2','166Er_pNFkB','151Eu_pp38', '155Gd_pS6','153Eu_pSTAT1', '154Sm_pSTAT3','174Yb_HLADR']
perio_stim_list_=['P._gingivalis']
perio_cell_list_=['Granulocytes_(CD45-CD66+)','B-Cells_(CD19+CD3-)','Classical_Monocytes_(CD14+CD16-)','MDSCs_(lin-CD11b-CD14+HLADRlo)','mDCs_(CD11c+HLADR+)','pDCs(CD123+HLADR+)','Intermediate_Monocytes_(CD14+CD16+)','Non-classical_Monocytes_(CD14-CD16+)','CD56+CD16-_NK_Cells','CD56loCD16+NK_Cells','NK_Cells_(CD7+)','CD4_T-Cells','Tregs_(CD25+FoxP3+)','CD8_T-Cells','CD8-CD4-_T-Cells']
cell_type='Classical Monocytes (CD14+CD16-)'
stim='P._gingivalis'
if [stim, cell_type] not in [['P._gingivalis', 'Non-classical_Monocytes_(CD14-CD16+)'],['P._gingivalis', 'NK_Cells_(CD7+)'],['TNFa', 'Granulocytes_(CD45-CD66+)']]:
    for marker in marker_list:
        if stim=='P._gingivalis':
            doms_stim='LPS'
        else:
            doms_stim=stim
    
        
        path_cohort_1='pred_surge_corrected_only_unstim.h5ad'
        output_path = f"plot_surge_dbl_corr_uns/{doms_stim}_{cell_type}_{marker}.png"
        
        plot_result(path_cohort_1, marker, output_path,doms_stim)
        print(f"Plot {marker} for {cell_type} and {doms_stim}")

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


Plot 149Sm_pCREB for Classical Monocytes (CD14+CD16-) and LPS


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


Plot 167Er_pERK12 for Classical Monocytes (CD14+CD16-) and LPS


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


...

Plot 150Nd_pSTAT5 for Classical Monocytes (CD14+CD16-) and LPS


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


Plot 168Yb_pSTAT6 for Classical Monocytes (CD14+CD16-) and LPS


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


Plot 174Yb_HLADR for Classical Monocytes (CD14+CD16-) and LPS
Plot 169Tm_CD25 for Classical Monocytes (CD14+CD16-) and LPS
